创建文章的倒排索引表，供后续计算使用
algo这部分开始，除非情况特殊，不再严格限制使用大数据的工具

In [2]:
import jieba
jieba.enable_paddle()
[",".join(jieba.cut_for_search("测试测试测试"))]

Installing paddle-tiny, please wait a minute......
Paddle enabled successfully......
Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\d\AppData\Local\Temp\jieba.cache
Loading model cost 0.880 seconds.
Prefix dict has been built successfully.


['测试,测试,测试']

In [57]:
from happybase import Table
# 从hbase中读取所有content的字段
import happybase
import json
def fresh_hbase(table="videobase_all"):
    conn = happybase.Connection('192.168.17.150', autoconnect=True)
    conn.open()
    table = conn.table(table)
    return table,conn
table,conn = fresh_hbase()
table.families()

{b'd': {'name': b'd:',
  'max_versions': 1,
  'compression': b'NONE',
  'in_memory': False,
  'bloom_filter_type': b'NONE',
  'bloom_filter_vector_size': 0,
  'bloom_filter_nb_hashes': 0,
  'block_cache_enabled': False,
  'time_to_live': 2147483647}}

In [27]:
# 对正文分词，并填入hbase中
for row,val in table.scan(columns=["d:content"]):
    row_str = row.decode()
    val_str = ",".join([ val[c].decode() for c in val ])
    toks = jieba.cut_for_search(val_str)
    toks = list(filter(lambda x: len(x)>1, toks))
    table.put(row=row_str,
              data={"d:toks":",".join(toks)},
              timestamp=2,
              wal=False)

In [10]:
# 查看分词的结果
for row,val in table.scan(limit=10, columns=["d:toks"]):
    val_str = ",".join([ (c.decode()+": "+val[c].decode()) for c in val ])
    print(str(row,encoding="utf8"),val_str)

400015692 d:toks: 京华,时报,时报讯,中国,菲律宾,大使,使馆,大使馆,近日,再次,提醒,中国,公民,前往,菲律宾,三宝,三宝颜,帕嘎蒂安市,巴托,风险,高风险,地区,中国,驻菲,大使,使馆,大使馆,最新,发布,风险,提示,须知,信息,近日,菲律宾,南部,伊沙,贝尔,再次,发生,再次发生,绑架,案件,包括,伊沙,贝尔,在内,南部,政府,菲政府,确定,最高,恐怖,威胁,风险,地区,中国,驻菲,大使,使馆,大使馆,以上,地区,可能,发生,爆炸,发生爆炸,绑架,枪击,纵火,恐怖,袭击,恐怖袭击,案件,确保,保安,安全,确保安全,中国,菲律宾,大使,使馆,大使馆,提醒,中国,公民,前往,上述,最高,恐怖,威胁,风险,地区,地区,中国,公民,采取,严密,安保,措施,及早,离开,中国,公民,谨慎,前往,恐怖,威胁,风险,地区,地区,采取,有效,采取有效,安全,防范,措施,防范措施,避免,前往,人多,复杂,场所,避免,夜间,外出,紧急,急情,情况,紧急情况,报警,联系,中国,驻菲使,领馆,大使,使馆,大使馆,0063,9178972695,宿务,领馆,总领馆,0063,9175495614,拉瓦,领事,领事馆,0063,9178051226,最高,恐怖,威胁,风险,地区,三宝,三宝颜,ZamboangaCity,帕嘎蒂安市,PagadianCity,巴托,CotabatoCity,古隆,TacurongCity,科嵘,纳塔市,KoronadalCity,托斯,桑托斯,将军,GeneralSantosCity,拉米,坦市,LamitanCity,伊沙,贝尔,IsabelCity,马京,达瑙省,MaguindanaoProvince,安帕,图安镇,AmpatuanTown,布鲁安镇,BuluanTown,苏禄,SuluProvince,迪奥,迪奥普,迪奥,迪奥普,Tiop,TiopTown,霍洛镇,JoloTown,达南,IndananTown,帕特,库尔,PatkulTown,菲较,恐怖,威胁,风险,地区,菲律宾,首都,地区,尼拉,马尼拉,MetroManila,NCR,南部,卡加,德奥,罗市,CagayanDeOroCity,塔巴,KidapawanCity,中旅,总社,中旅总社,暂停,旅游,业务,确保,中国,公民,出境,旅游,人身,财产,安全,中旅,总社,中旅

In [21]:
# 根据toks计算每个文章的toks_count和全局的toks_count，并写入hbase
import collections as c
table,conn = fresh_hbase()
all_toks_count = c.Counter()
for row,val in table.scan(columns=["d:toks"]):
    val_str = ("".join([ val[c].decode() for c in val ])).split(",")
    toks_count = c.Counter(val_str)
    all_toks_count += toks_count
    toks_count_col = ",".join([ str(c[0]) + ":" + str(c[1]) for c in toks_count.most_common() ])
    # table.put(row=row.decode(),
    #           data={"d:toks_count": toks_count_col},
    #           timestamp=3,
    #           wal=False)
print("done")

done


In [49]:
# 检查写入情况
for row,val in table.scan(columns=["d:toks_count"],limit=1):
    val_str = ",".join([ (c.decode()+": "+val[c].decode()) for c in val ])
    print(str(row,encoding="utf8"),val_str)

400015692 d:toks_count: 中国:10,地区:10,风险:7,中旅:7,总社:7,中旅总社:7,恐怖:6,菲律宾:5,大使:5,使馆:5,大使馆:5,公民:5,威胁:5,旅游:5,前往:4,近日:3,南部:3,伊沙:3,贝尔:3,最高:3,安全:3,0063:3,再次:2,提醒:2,三宝:2,三宝颜:2,帕嘎蒂安市:2,巴托:2,驻菲:2,发生:2,绑架:2,案件:2,确保:2,采取:2,措施:2,避免:2,领馆:2,迪奥:2,迪奥普:2,暂停:2,越南:2,京华:1,时报:1,时报讯:1,高风险:1,最新:1,发布:1,提示:1,须知:1,信息:1,再次发生:1,包括:1,在内:1,政府:1,菲政府:1,确定:1,以上:1,可能:1,爆炸:1,发生爆炸:1,枪击:1,纵火:1,袭击:1,恐怖袭击:1,保安:1,确保安全:1,上述:1,严密:1,安保:1,及早:1,离开:1,谨慎:1,有效:1,采取有效:1,防范:1,防范措施:1,人多:1,复杂:1,场所:1,夜间:1,外出:1,紧急:1,急情:1,情况:1,紧急情况:1,报警:1,联系:1,驻菲使:1,9178972695:1,宿务:1,总领馆:1,9175495614:1,拉瓦:1,领事:1,领事馆:1,9178051226:1,ZamboangaCity:1,PagadianCity:1,CotabatoCity:1,古隆:1,TacurongCity:1,科嵘:1,纳塔市:1,KoronadalCity:1,托斯:1,桑托斯:1,将军:1,GeneralSantosCity:1,拉米:1,坦市:1,LamitanCity:1,IsabelCity:1,马京:1,达瑙省:1,MaguindanaoProvince:1,安帕:1,图安镇:1,AmpatuanTown:1,布鲁安镇:1,BuluanTown:1,苏禄:1,SuluProvince:1,Tiop:1,TiopTown:1,霍洛镇:1,JoloTown:1,达南:1,IndananTown:1,帕特:1,库尔:1,PatkulTown:1,菲较:1,首都:1,尼拉:1,马尼拉:1,MetroManila:1,NCR:1,卡加:1,德奥:1,罗市:1,CagayanDeOroCity:1,塔巴:1,KidapawanCity:1,业

In [34]:
print(len(all_toks_count))
print(all_toks_count.most_common())

63143
[('一个', 1477), ('自己', 1349), ('中国', 1319), ('公司', 1307), ('可以', 1233), ('我们', 1093), ('没有', 932), ('市场', 862), ('孩子', 831), ('工作', 805), ('时间', 784), ('10', 759), ('就是', 729), ('企业', 691), ('进行', 681), ('他们', 675), ('问题', 669), ('发展', 657), ('已经', 656), ('如果', 633), ('可能', 631), ('投资', 625), ('这个', 614), ('什么', 598), ('锦绣', 593), ('管理', 544), ('记者', 539), ('这样', 533), ('因为', 532), ('不是', 528), ('经济', 524), ('关注', 518), ('通过', 490), ('产品', 489), ('人民', 489), ('情况', 485), ('开始', 484), ('美国', 483), ('同时', 481), ('大学', 481), ('服务', 466), ('国家', 461), ('安全', 457), ('需要', 453), ('生活', 451), ('第一', 451), ('技术', 444), ('方面', 437), ('时候', 437), ('一些', 430), ('对于', 429), ('重要', 429), ('还是', 427), ('表示', 425), ('信息', 420), ('目前', 417), ('发现', 412), ('数据', 411), ('提供', 410), ('世界', 409), ('很多', 407), ('但是', 404), ('银行', 402), ('社会', 401), ('人员', 400), ('所以', 395), ('成为', 393), ('大家', 393), ('这些', 391), ('教育', 388), ('作为', 382), ('使用', 382), ('活动', 377), ('影响', 377), ('设计', 370), ('能力', 368),

一共有六万多个词，我们进行一下截断再看看

In [50]:
all_toks_count_cut = []
for w in all_toks_count.most_common():
    if w[1]<=1 or len(w[0].strip())<=1:
        #print(w)
        continue
    all_toks_count_cut.append(w)
len(all_toks_count_cut)

31333

In [51]:
# 按照截断后的词，构建反向索引的表
import collections as c
reverse_word_table = {}
for word in all_toks_count_cut:
    reverse_word_table[word[0]]=[]
table,conn = fresh_hbase()
for row,val in table.scan(columns=["d:toks_count"]):
    val_str = ("".join([ val[c].decode() for c in val ])).split(",")
    for word in val_str:
        [word_str, word_count] = word.split(":")
        #print(word_str,word_count)
        if word_str in reverse_word_table:
            reverse_word_table[word_str].append((row.decode(), int(word_count)))
for word in reverse_word_table:
    reverse_word_table[word].sort(key=lambda x: x[1], reverse=True)
print(reverse_word_table['世界'])

[('401061958', 24), ('471409579', 14), ('410963540', 11), ('410428937', 10), ('470308283', 10), ('470495789', 10), ('471044784', 10), ('470232882', 9), ('400097880', 8), ('470399717', 8), ('471717457', 8), ('460063115', 7), ('471125743', 7), ('401050637', 6), ('410863694', 6), ('471102017', 6), ('471662611', 6), ('471675924', 6), ('460028881', 5), ('471514554', 5), ('401167836', 4), ('461095668', 4), ('470358667', 4), ('470400561', 4), ('470451828', 4), ('470809668', 4), ('470842682', 4), ('470899479', 4), ('471617244', 4), ('471781338', 4), ('410164104', 3), ('410302787', 3), ('410731192', 3), ('410746000', 3), ('410773938', 3), ('410830996', 3), ('410909880', 3), ('411166070', 3), ('411276109', 3), ('461008829', 3), ('461186259', 3), ('470114085', 3), ('470247470', 3), ('470262695', 3), ('470616136', 3), ('470822172', 3), ('471441734', 3), ('410203964', 2), ('410217335', 2), ('410245117', 2), ('410380081', 2), ('410589430', 2), ('410662758', 2), ('410794757', 2), ('410948425', 2), ('

In [71]:
# 构建倒排表
table,conn = fresh_hbase()
conn.create_table("news_invert", {'d': dict(max_versions=1, block_cache_enabled=False)})
conn.close()

In [3]:
from pyhive import hive
cursor = hive.connect("192.168.17.150").cursor()
hive_sql = """
    CREATE EXTERNAL TABLE IF NOT EXISTS resys_one.news_invert (id STRING, invert STRING)
    STORED BY 'org.apache.hadoop.hive.hbase.HBaseStorageHandler'
    WITH SERDEPROPERTIES (\"hbase.columns.mapping\" = \"d:invert\")
    TBLPROPERTIES(\"hbase.table.name\" = \"news_invert\")
"""
cursor.execute(hive_sql)
cursor.close()

In [53]:
# 写入倒排表
table,conn = fresh_hbase("news_invert")
for word,i in zip(reverse_word_table,range(1,200000)):
    row = word
    val = ",".join([w[0]+":"+str(w[1]) for w in reverse_word_table[word]])
    table.put(row=row,
              data={"d:invert": val},
              timestamp=4,
              wal=False)
conn.close()

In [58]:
# 检查写入情况
for row,val in conn.table("news_invert").scan(columns=["d:invert"],limit=1):
    val_str = ",".join([ (c.decode()+": "+val[c].decode()) for c in val ])
    print(str(row,encoding="utf8"),val_str)


-- d:invert: 470741171:10,470505973:4,470745270:4,470611036:2,471140753:2,471335165:2,471790063:2,410063596:1,410185697:1,410616344:1,410933104:1,470284706:1,470300594:1,470576318:1,471310470:1


In [59]:
print("done")


done
